In [344]:
import re
import time, datetime
from collections import Counter

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import json
import string

In [345]:
data = pd.read_json("flavor-data.json")
datalist = data['drinks'].tolist()


In [346]:

X = pd.DataFrame(datalist)
y = X.pop('flavors')
print(X.shape, y.shape)

t = [x for row in y for x in row]
flavors = Counter(t)
print(flavors, len(flavors))


(609, 2) (609,)
Counter({'Sweet': 399, 'Fruity': 162, 'Sour': 58, 'Spicy': 33, 'Savory': 26, 'Hot': 22, 'Frozen': 10, 'Beginner': 7}) 8


In [347]:
# data preprocessing
yBin = mlb.fit_transform(y)
print(yBin)
xTr, xTe, yTr, yTe = train_test_split(X, yBin, test_size=0.10)

print(xTr.shape, yTr.shape)
print("Loaded:","xTr",xTr.shape,"yTr",yTr.shape,"xTe",xTe.shape)
assert(xTr.shape[1] == xTe.shape[1])

def preprocess_meta(xTr):
    X = xTr.dropna(axis=1)
    # X.loc[:,('created')] = pd.to_datetime(X.loc[:,'created']).dt.time.apply(lambda x: x.hour*60 + x.minute)
    # X.loc[:,('has_link')] = X.loc[:,'text'].str.contains('http').astype(int)
    X.loc[:,('ingredientCount')] = [ len(x) for x in X.loc[:,'ingredients'] ]
        
    return X[['ingredientCount']]

# remove/replace useless text
def preprocess_text(X):
    # ingredient list to str and treat name as part of same doc
    i = [ str(x[1]['name'] + ' ' + ' '.join(x[1]['ingredients'])) for x in X.iterrows() ]
    
    return np.array(i)

# Preprocess the text data: get_text_data
get_text_data = FunctionTransformer(
    lambda x: preprocess_text(x),
    validate=False
)
# Preprocess the numeric data: get_numeric_data
get_meta_data = FunctionTransformer(
    # use useful metadata and create/modify features from xTr cols
    lambda x: preprocess_meta(x),
    validate=False)
print("Final Metadata Columns:",list(get_meta_data.transform(xTr)))
print("Processed:",
      "xTr_text",get_text_data.transform(xTr).shape,
      "xTr_meta",get_meta_data.transform(xTr).shape,
      "yTr",yTr.shape)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 1 1 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]
(548, 2) (548, 8)
Loaded: xTr (548, 2) yTr (548, 8) xTe (61, 2)
Final Metadata Columns: ['ingredientCount']
Processed: xTr_text (548,) xTr_meta (548, 1) yTr (548, 8)


In [354]:
# model configuration

# vectorize unlabeled testing and training data text
vectorizer = TfidfVectorizer(
    analyzer='word',
    norm=None,
)

# default model
default_model = LogisticRegression(solver='lbfgs', max_iter=1000); #best(?)
# default_model = LinearSVC(max_iter=100000);
# default_model = BernoulliNB()
# default_model = RandomForestClassifier()

In [355]:
# add more features

start_time=time.time()
pipe = Pipeline([
    ('union', FeatureUnion( # add both text and metadata to xTr
        transformer_list = [
            ('numeric_features', Pipeline([
                ('selector', get_meta_data)
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('vectorizer', vectorizer)
            ]))
        ]
    )),
    ('clf', OneVsRestClassifier(default_model))
])

param_grid = {'union__text_features__vectorizer__max_features': [10000, 30000],
              # 'clf__estimator__C': [0.1, 1]
             } 
grid = GridSearchCV(pipe, param_grid, cv=6, scoring='f1_samples')
grid4 = grid.fit(xTr, yTr)

print('Accuracy score:',grid4.best_score_)
print('Best params:',grid4.best_params_)
print('Estimator:',grid4.estimator)

end_time=time.time()
print("Pipeline time:",end_time-start_time)


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in s

Accuracy score: 0.6109489051094891
Best params: {'union__text_features__vectorizer__max_features': 10000}
Estimator: Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('numeric_features', Pipeline(memory=None,
     steps=[('selector', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x12c24ab70>, inv_kw_args=None,
          inverse_func=None, kw_ar...e=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None))])
Pipeline time: 5.085464954376221


In [356]:
pred = grid4.predict(xTe)
pred_inv = mlb.inverse_transform(pred)
actual = mlb.inverse_transform(yTe)
xTe = xTe.assign(label = pred_inv)
xTe = xTe.assign(actual = actual)
print(xTe[:10])


precision, recall, fscore, support = metrics.precision_recall_fscore_support(yTe, pred, average='weighted')
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('f1score: {}'.format(fscore))
print('support: {}'.format(support))


                                           ingredients                 name  \
549  [1 part Pinnacle® Citrus Vodka, 2 parts Lemona...         Limonlicious   
371  [1 part JDK & Sons™ Crave Chocolate Truffle Li...   Banana Truffle-nog   
152  [1 part DeKuyper® Peachtree® Schnapps Liqueur,...            Nectarita   
245  [1 part Midori® Melon Liqueur, 1 part Pinnacle...        Spider's Kiss   
284  [1 part Jim Beam® Devil's Cut®, 1 part Jim Bea...            The Rider   
161  [1 part DeKuyper® Peppermint Schnapps, Hot Cho...  Christmas Mint Kiss   
282  [1 part Pinnacle® Original Vodka, 2 parts Cran...          Flirty Fizz   
417  [1 part Cruzan® Aged Light Rum, 1 part DeKuype...  Strawberry Daiquiri   
234  [1 part DeKuyper® Creme de Menthe Green Liqueu...      Mintini Martini   
428  [1 part Hornitos® Plata Tequila, 1 Lime Wedges...        The Luchadora   

         label           actual  
549   (Sweet,)         (Sweet,)  
371   (Sweet,)         (Sweet,)  
152   (Sweet,)          (Sou

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [314]:
# run on actual drink list
with open('../app/static/drinks-with-related.json', 'r') as infile:  
    jsonData = json.load(infile)
    drinklst = jsonData['drinks']
    drinks = pd.DataFrame(drinklst)
    
    pred = grid4.predict(drinks)
    pred = mlb.inverse_transform(pred)
    
    drinks['tags'] = pred
    print(drinks[:10])
    

        categories                                        description  \
0               []  Place one ice cube in the glass and add 1 1/2 ...   
1               []  Fill a pint glass with ice. Pour vodka and cof...   
2      [halloween]  Shake with ice and strain into chilled cocktai...   
3               []  Bring all syrup ingredients to a boil in a sma...   
4               []  In a blender, combine ice, rum, limeade concen...   
5               []  Stir the orange juice, pineapple juice, simple...   
6               []  Muddle the sugar into the lime wedges in an ol...   
7               []  Cut half a lime into pieces, place in a shaker...   
8      [afternoon]  Put ice cubes in collins glass and add lime vo...   
9  [party, sports]  Rub rim of cocktail glass with rind of lemon o...   

                                         ingredients  \
0  [1 1/2 oz  Advocaat, 8-10 oz cold  Lemonade, 1...   
1  [ice cubes, or as needed, 2 fluid ounces vodka...   
2  [1 oz  Blanco tequila, 3/

In [315]:
jsondata = {}
jsondata['drinks'] = []
for i, drink in drinks.iterrows():
    try:
        ing = [(b +" " + a) for (a,b) in drink['ingredients'] if a != ""]
    except ValueError:
        ing = [a for a in drink['ingredients'] if a != ""]
        
    if 'instructions' in drink:
        drink['description'] = drink['instructions']
    
    jsondata['drinks'].append({
        'name' : drink['name'],
        'description' : drink['description'],
        'src' : drink['src'],
        'ingredients' : ing,
        'rating' : drink['rating'],
        'reviews' : drink.get('reviews', []),
        'categories' : drink.get('categories', []),
        'related' : drink.get('related', []),
        'tags' : drink.get('tags', [])
    })

In [316]:
with open('../app/static/drinks-with-related-and-tags.json', 'w') as outfile:  
    json.dump(jsondata, outfile)